In [ ]:
import csv
import logging
from db import DB

# Initialize your database connection (assuming DB is your ORM class)
db = DB()

# Path to your new CSV file
csv_file = 'data/2025_GAAP_Taxonomy.csv'

def load_csv(file_path):
    """
    Loads the data from a CSV file.

    Parameters:
    - file_path (str): Path to the CSV file.

    Returns:
    - list: Parsed data from the CSV file as a list of dictionaries.
    """
    data = []
    with open(file_path, 'r') as f:
        reader = csv.DictReader(f)
        for row in reader:
            data.append(row)
    return data

def upsert_gaap_taxonomy(db, csv_data):
    """
    Upserts the GAAP taxonomy data into the database.

    Parameters:
    - db: The DB instance from your ORM.
    - csv_data (list): The parsed CSV data.
    """
    try:
        for row in csv_data:
            # Note: Other filtering conditions could be added here as well
            # TODO: Resolve mapped types
            if row['prefix'] != "us-gaap" or row['name'].endswith("Text") or not row['type'].startswith("xbrli:") and row['type'] != "dtr-types:perShareItemType":
                continue

            name = row['name']
            concept_type = row['type']
            balance = row['balance'] if row['balance'] else None
            period_type = row['periodType'] if row['periodType'] else None
            label = row['label'] if row['label'] else None
            documentation = row['documentation'] if row['documentation'] else None

            concept_type_id = db.upsert_entity('us_gaap_concept_type', {'concept_type': concept_type}, ['concept_type'])

            # Upsert balance type if provided
            if balance is not None:
                balance_type_id = db.upsert_entity('us_gaap_balance_type', {'balance': balance}, ['balance'])
            else:
                balance_type_id = None
            
            # Upsert period type if provided
            if period_type is not None:
                period_type_id = db.upsert_entity('us_gaap_period_type', {'period_type': period_type}, ['period_type'])
            else:
                period_type_id = None
            
            # Upsert the concept itself in the `us_gaap_concept` table
            concept_data = {
                'name': name,
                'concept_type_id': concept_type_id,
                'balance_type_id': balance_type_id,
                'period_type_id': period_type_id,
                'label': label,
                'documentation': documentation,
            }

            concept_id = db.upsert_entity('us_gaap_concept', concept_data, ['name'])

            logging.debug(f"Upserted data for concept ({concept_id}): {name}")

        logging.info('GAAP taxonomy data has been successfully upserted.')
    except Exception as e:
        logging.error(f"Error upserting GAAP taxonomy data: {e}")
        raise

if __name__ == "__main__":
    # Load the CSV data
    csv_data = load_csv(csv_file)

    # Upsert the data into the database
    upsert_gaap_taxonomy(db, csv_data)
